# State space representation

The "standard" or most commonly used state space representation is 

\begin{align} \dot{x} &= Ax + Bu \\ y &= Cx + Du \end{align}

Take note that Seborg uses a slightly different version:

\begin{align} \dot{x} &= Ax + Bu + Ed\\ y &= Cx \end{align}

This second version can not represent pure gain systems as it effectively assumes $D=0$. It is also possible to stack $u$ and $d$ from the bottom form into one input vector, so the $E$ matrix really doesn't add much. As you may infer, I prefer the top version and it is also the version used by most libraries.

In [1]:
import numpy
import numpy.linalg

## Converting between state space and transfer function forms

### Scipy.signal

The `scipy.signal` library handles conversion between transfer function coefficients and state space matrices easily:

In [2]:
import scipy.signal

In [3]:
G = scipy.signal.lti(1, [1, 1])

In [4]:
G

TransferFunctionContinuous(
array([1.]),
array([1., 1.]),
dt: None
)

This object allows us to access the numerator and denominator

In [5]:
G.num, G.den

(array([1.]), array([1., 1.]))

To convert to state space, we can use the `.to_ss()` method

In [6]:
Gss = G.to_ss()

In [7]:
Gss.A, Gss.B, Gss.C, Gss.D

(array([[-1.]]), array([[1.]]), array([[1.]]), array([[0.]]))

We can build another object using the state space matrices instead of the Laplace form

In [8]:
G2ss = scipy.signal.lti(Gss.A, Gss.B, Gss.C, Gss.D)
G2ss

StateSpaceContinuous(
array([[-1.]]),
array([[1.]]),
array([[1.]]),
array([[0.]]),
dt: None
)

We can convert to transfer function form using `.to_tf()` (there is a small warning about bad coefficients, but the answer is reliable).

In [9]:
G2 = G2ss.to_tf()

/Users/alchemyst/anaconda3/lib/python3.6/site-packages/scipy/signal/filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)


We can now access the numerator and denominator again:

In [10]:
G2.num, G2.den

(array([1.]), array([1., 1.]))

Instead of building objects we can also use the functions in `scipy.signal.lti_conversion`:

In [11]:
scipy.signal.lti_conversion.tf2ss(1, [1, 1])

(array([[-1.]]), array([[1.]]), array([[1.]]), array([[0.]]))

In [12]:
scipy.signal.lti_conversion.ss2tf(-1, 1, 1, 0)

(array([[0, 1]]), array([1., 1.]))

### Control library

The control library is supposed to handle this pretty easily as well:

**Note** There appears to be an error in the control library (which I have reported [here](https://github.com/python-control/python-control/issues/156)) when converting the examples I used in the `scipy.signal` section. My recommendation is to use that library if you see errors when using the control library.

In [13]:
import control

In [14]:
Gtf = control.tf([1, 1], [1, 2, 1, 1])
Gtf


       s + 1
-------------------
s^3 + 2 s^2 + s + 1

In the control library we convert the system using `ss` (short for state space) to get a State Space representation:

In [15]:
Gss = control.ss(Gtf)
Gss

A = [[-3.88578059e-16 -2.31310922e-17  1.00000000e+00]
 [ 1.00000000e+00  1.72935487e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.00000000e+00]]

B = [[1.]
 [1.]
 [0.]]

C = [[ 0.  0. -1.]]

D = [[0.]]

In [16]:
Gss.A

matrix([[-3.88578059e-16, -2.31310922e-17,  1.00000000e+00],
        [ 1.00000000e+00,  1.72935487e-16,  1.00000000e+00],
        [ 0.00000000e+00, -1.00000000e+00, -2.00000000e+00]])

## Symbolic conversion

It is easy to convert state space models to transfer functions since the Laplace transform is a linear operator:

$$ \dot{x} = Ax + Bu \quad \therefore \quad sX(s) = AX(s) + BU(s) \quad X(s) = (sI - A)^{-1}BU(s)$$
$$ y = Cx + Du \quad \therefore \quad Y(s) = CX(s) + DU(s) \quad Y(s) = \underbrace{(C(sI - A)^{-1}B + D)}_{G(s)}U(s)$$


In [17]:
import sympy

In [18]:
s = sympy.Symbol('s')

In [19]:
A, B, C, D = [sympy.Matrix(m) for m in [G2ss.A, G2ss.B, G2ss.C, G2ss.D]]

In [20]:
A, B, C, D

(Matrix([[-1.0]]), Matrix([[1.0]]), Matrix([[1.0]]), Matrix([[0.0]]))

In [21]:
I = sympy.eye(A.shape[0])

In [22]:
G = C*(s*I - A).inv()*B + D
sympy.nsimplify(G[0,0]).simplify()

1/(s + 1)

## Analysis

Notice that the roots of the characteristic function correspond with the eigenvalues of the A matrix. The numerator and denominator of control transfer functions are stored as lists of lists to accomodate MIMO systems.

In [23]:
Gtf.pole()

array([-1.75487767+0.j        , -0.12256117+0.74486177j,
       -0.12256117-0.74486177j])

In [24]:
numpy.roots(Gtf.den[0][0])

array([-1.75487767+0.j        , -0.12256117+0.74486177j,
       -0.12256117-0.74486177j])

In [25]:
numpy.linalg.eig(Gss.A)

(array([-0.12256117+0.74486177j, -0.12256117-0.74486177j,
        -1.75487767+0.j        ]),
 matrix([[-0.26086225-0.43441848j, -0.26086225+0.43441848j,
          -0.48423834+0.j        ],
         [-0.77260237+0.j        , -0.77260237-0.j        ,
          -0.20829982+0.j        ],
         [ 0.3555533 -0.14106348j,  0.3555533 +0.14106348j,
           0.84977904+0.j        ]]))